## Annotation Extraction 

In [13]:
from jupyterthemes import jtplot
import pandas as pd
import numpy as np
import wfdb
from wfdb import processing
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

In [14]:
def annotation_symbols(str1,str2):
    ann = wfdb.rdann(str1+str2, 'atr')
    ann_list = np.array(ann.aux_note)
    return np.array(ann_list)

def annotation_labels(str1,str2):
    ann = wfdb.rdann(str1+str2, 'atr')
    ann_list = np.array(ann.symbol)
    return np.array(ann_list)

def annotation_locations(str1,str2):
    ann = wfdb.rdann(str1+str2, 'atr')
    ann_list = np.array(ann.sample)
    return np.array(ann_list)

In [24]:
ss = pd.read_csv("./Signal_Data/signals_anns.csv",sep=",")
ss = np.array(ss)

In [25]:
ann_list =[]
ann_locations = []
ann_labels = []

str1 = "./Data_thesis/MIT/"
MIT_str = ["100","101","102","103","104","105","106","107","108","109","111","112","113","114","115","116","117","118","119",
           "121","122","123","124","200","201","202","203","205","207","208","209","210","212","213","214","215","217",
           "219","220","221","222","223","228","230","231","232","233","234"]

for i in MIT_str:
    ann_list.append(annotation_symbols(str1,i))
    ann_locations.append(annotation_locations(str1,i))
    ann_labels.append(annotation_labels(str1,i))
    
str2 = "./Data_thesis/CUDB/"
cudb_str = ["cu01","cu02","cu03","cu04","cu05","cu06","cu07","cu08","cu09","cu10","cu11","cu12","cu13","cu14","cu15","cu16",
            "cu17","cu18","cu19","cu20","cu21","cu22","cu23","cu24","cu25","cu26","cu27","cu28","cu29","cu30","cu31","cu32",
            "cu33","cu34","cu35"]

for i in cudb_str:
    ann_list.append(annotation_symbols(str2,i))
    ann_locations.append(annotation_locations(str2,i))
    ann_labels.append(annotation_labels(str2,i))
   
str3 = "./Data_thesis/VFDB/"
vfdb_str = ["418","419","420","421","422","423","424","425","426","427","428","429","430","602","605","607",
            "609","610","611","612","614","615"]

for i in vfdb_str:
    ann_list.append(annotation_symbols(str3,i))
    ann_locations.append(annotation_locations(str3,i))
    ann_labels.append(annotation_labels(str3,i))

In [26]:
ann_list=np.array(ann_list)
ann_locations = np.array(ann_locations)
ann_labels = np.array(ann_labels)

In [27]:
wfdb.io.show_ann_labels()

    label_store symbol                                    description
0             0                              Not an actual annotation
1             1      N                                    Normal beat
2             2      L                  Left bundle branch block beat
3             3      R                 Right bundle branch block beat
4             4      a                Aberrated atrial premature beat
5             5      V              Premature ventricular contraction
6             6      F          Fusion of ventricular and normal beat
7             7      J              Nodal (junctional) premature beat
8             8      A                   Atrial premature contraction
9             9      S     Premature or ectopic supraventricular beat
10           10      E                        Ventricular escape beat
11           11      j                 Nodal (junctional) escape beat
12           12      /                                     Paced beat
13           13     

In [28]:
wfdb.io.show_ann_classes()

     extension                        description  human_reviewed
atr        atr          Reference ECG annotations            True
blh        blh         Human reviewed beat labels            True
blm        blm                Machine beat labels           False
alh        alh              Human reviewed alarms            True
alm        alm                     Machine alarms           False
qrsc      qrsc      Human reviewed qrs detections            True
qrs        qrs             Machine QRS detections           False
bph        bph  Human reviewed BP beat detections            True
bpm        bpm         Machine BP beat detections           False


In [29]:
def fib_location_VF(lis,loc,lis2):
    listlist = []
    listmist = []
    for i in range(len(lis)):
        for j in range(len(lis[i])):
            if lis[i][j] == '(VF' or lis[i][j]=='(VFL' or lis[i][j] == '(VFIB':
                for k in range(len(lis[i][j+1:-1])):
                    if lis[i][j+k+1] != '':
                        lastone = loc[i][j+k]
                        duration = lastone - loc[i][j-1]
                        break
                if i <=47:
                    listlist.append(((i)*len(ss[i]))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append(((i)*len(ss[i]))+int(loc[i][j+k]/len(ss[i][0])))
                elif 47 < i < 83:
                    listlist.append((((i-47)*len(ss[i]))+(196*47))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append((((i-47)*len(ss[i]))+(196*47))+int(loc[i][j+k]/len(ss[i][0])))
                elif i >= 83:
                    listlist.append((((i-82)*len(ss[i]))+(196*47)+(38*34))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append((((i-82)*len(ss[i]))+(196*47)+(38*34))+int(loc[i][j+k]/len(ss[i][0])))
                    
    for u in range(len(lis2)):
        for v in range(len(lis2[u])):
            if lis2[u][v] == '[':
                for qq in range(len(lis2[u][v+1:-1])):
                    if lis2[u][v+qq] == ']':
                        lastone = loc[u][v+qq]
                        duration = lastone - loc[u][v-1]
                        break
                    else:
                        break
                if u <=47:
                    listlist.append(((u)*len(ss[u]))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append(((u)*len(ss[u]))+int(loc[u][v+qq]/len(ss[u][0])))
                elif 47 < u < 83:
                    listlist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v+qq]/len(ss[u][0])))
                elif u >= 83:
                    listlist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v+qq]/len(ss[u][0])))
            elif lis2[u][v] == '!':
                for nn in range(len(lis2[u][v+1:-1])):
                    if lis2[u][v+nn+1] != '!' and lis2[u][v+nn+1] != ['"', '+', '/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S','V', '[', 
                                                                      ']', 'a', 'e', 'f', 'j', 'x', '|', '~']:
                        lastone = loc[u][v+nn]
                        duration = lastone - loc[u][v-1]
                        break
                if u <=47:
                    listlist.append(((u)*len(ss[u]))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append(((u)*len(ss[u]))+int(loc[u][v+nn]/len(ss[u][0])))
                elif 47 < u < 83:
                    listlist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v+nn]/len(ss[u][0])))
                elif u >= 83:
                    listlist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v]/len(ss[u][0])))
                    listmist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v+nn]/len(ss[u][0])))
    return np.array(listlist),np.array(listmist)

In [63]:
sm = pd.read_csv("./Signal_Data/signals.csv",sep = ",")

In [66]:
del sm['Unnamed: 0']

In [75]:
np.array(sm)[0

'['

In [30]:
VF_anns = fib_location_VF(ann_list,ann_locations,ann_labels)

TypeError: object of type 'int' has no len()

1. 3 Premature ventricular contractions = VT
2. ! = VFL
3. Square brackets (open and closed) = VF
4. R on T premature contracitons = VT

In [ ]:
def fib_location_VT(lis,loc,lis2):
    listlist = []
    listmist = []
    for i in range(len(lis)):
        for j in range(len(lis[i])):
            if lis[i][j] == '(SVTA' or lis[i][j]=='(VT':
                for k in range(len(ann_list[i][j+1:-1])):
                    if ann_list[i][j+k+1] != '':
                        lastone = loc[i][j+k]
                        duration = lastone - loc[i][j-1]
                        break
                if i <=47:
                    listlist.append(((i)*len(ss[i]))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append(((i)*len(ss[i]))+int(loc[i][j+k]/len(ss[i][0])))
                elif 47 < i < 83:
                    listlist.append((((i-47)*len(ss[i]))+(196*47))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append((((i-47)*len(ss[i]))+(196*47))+int(loc[i][j+k]/len(ss[i][0])))
                elif i >= 83:
                    listlist.append((((i-82)*len(ss[i]))+(196*47)+(38*34))+int(loc[i][j]/len(ss[i][0])))
                    listmist.append((((i-82)*len(ss[i]))+(196*47)+(38*34))+int(loc[i][j+k]/len(ss[i][0])))
                
    for u in range(len(lis2)):
        cc = 0
        for v in range(len(lis2[u])):
            if lis2[u][v] == 'V':
                cc +=1
            else:
                if v == len(lis2[u])-1:
                    cc = 0
                elif lis2[u][v+1] ==  ['!', '"', '+', '/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S','V', '[', ']', 'a', 'e', 'f', 'j', 'x', '|', '~']:#and lis2[u][v+2]=='+':
                    cc +=1
            if cc ==4:
                for qq in range(len(lis2[u][v-4:-1])):
                    if lis2[u][v+qq] != 'V':
                        lastone = loc[u][v+qq]
                        duration = lastone - loc[u][v-1]
                        break
                    else:
                        break
                if u <=47:
                    listlist.append(((u)*len(ss[u]))+int(loc[u][v-3]/len(ss[u][0])))
                    listmist.append(((u)*len(ss[u]))+int(loc[u][v+qq-1]/len(ss[u][0])))
                elif 47 < u < 83:
                    listlist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v-3]/len(ss[u][0])))
                    listmist.append((((u-47)*len(ss[u]))+(196*47))+int(loc[u][v+qq-1]/len(ss[u][0])))
                elif u >= 83:
                    listlist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v-3]/len(ss[u][0])))
                    listmist.append((((u-82)*len(ss[u]))+(196*47)+(38*34))+int(loc[u][v+qq-1]/len(ss[u][0])))
                    
    return np.array(listlist),np.array(listmist)

In [ ]:
VT_anns = fib_location_VT(ann_list,ann_locations,ann_labels)

In [ ]:
def label_creator(anns): 
    labels = np.zeros(14236)
    labels = np.array(labels)
    i = 0
    counter = 0
    while 3 > 2:
        for l in range(anns[0][i],anns[1][i]+1):
            labels[l] = 1
            counter+=1
        i+=1
        if i == len(anns[0]):
            break
    return pd.DataFrame(labels)

In [ ]:
VF_labels = label_creator(VF_anns)

In [ ]:
counter1 = 0
for i in np.array(VF_labels):
    if i ==1:
        counter1+=1
counter1

In [ ]:
VT_labels = label_creator(VT_anns)

In [ ]:
counter2 = 0
for i in np.array(VT_labels):
    if i == 1:
        counter2 +=1
counter2

In [ ]:
counter1+counter2

In [ ]:
v = []
for i in range(len(ann_list)):
    for j in range(len(ann_list[i])):
        v.append(ann_list[i][j])
np.unique(np.array(v))

In [ ]:
v = []
for i in range(len(ann_labels)):
    for j in range(len(ann_labels[i])):
        v.append(ann_labels[i][j])
np.unique(np.array(v))

In [ ]:
VT = pd.DataFrame(VT_labels)
VT.columns = ["VT"]
VF = pd.DataFrame(VF_labels)
VF.columns = ["VF"]

In [ ]:
VF.to_csv("./Parameter_Data/VF_final.csv",sep=',')
VT.to_csv("./Parameter_Data/VT_final.csv",sep=',')